# Anime

Начнем с самого легкого, так как дальше нужно будет собирать целый набор данных. Удачи.

Вот документация: https://jikan.docs.apiary.io/#reference

А вот оболочка для API: https://github.com/abhinavk99/jikanpy

In [1]:
import jikanpy as jp
import pandas as pd
from time import sleep

In [2]:
anime_dataset = pd.read_csv('./datasets/anime_dataset/anime.csv')

In [3]:
ids = anime_dataset.MAL_ID.tolist()

In [4]:
jikan = jp.Jikan()

In [5]:
def get_cleared_recs(uncleared_recs):
    titles = []
    mal_ids = []
    i = 0
    while i != 7:
        try:
            title = uncleared_recs['recommendations'][i]['title']
            mal_id = uncleared_recs['recommendations'][i]['mal_id']
            titles.append(title)
            mal_ids.append(mal_id)
            i += 1
        except IndexError:
            titles.append('No')
            mal_ids.append('No')
            i += 1
    return titles, mal_ids

In [6]:
def get_recs(id):
    uncleared_recs = jikan.anime(id, extension='recommendations')
    recs = get_cleared_recs(uncleared_recs)
    return recs

In [13]:
i = 0
anime_recs = pd.DataFrame(columns=['ID', 'Reco_Title', 'Reco_ID'])
fake_ids = []
for item_id in ids:
    titles, mal_ids = get_recs(item_id)
    for i in range(7):
        anime_recs.loc[len(anime_recs.index)] = [item_id, titles[i], mal_ids[i]]
    if item_id % 100 == 0:
        print(item_id)
    sleep(4)

KeyboardInterrupt: 

ID объекта и название рекомендации, само собой, нужны, первый - для того, чтобы знать, к чему привязать, второй - для того, чтобы знать что мы привязываем. Зачем ID рекомендаций? Для того, чтобы получить их признаки.

In [ ]:
anime_recs.head()

In [ ]:
anime_recs.to_csv('C:/Users/ASDW/Python/Projects/Recosys 2.0/datasets/anime_dataset/recommendations.csv')

Количество уникальных значений делим на количество секунд в 1 часе.

In [14]:
17562*4/3600

19.513333333333332

Ы. Почти 5 часов непрерывного исполнения кода. Класс.

# Movies

Самый лучший API, так как не имеет ограничений, бесплатный и быстрый. 

Вот документация: https://developers.themoviedb.org/3/getting-started/introduction

А вот для него оболочка: https://github.com/celiao/tmdbsimple/

In [145]:
import json
import tmdbsimple as tmdb

In [146]:
tmdb.API_KEY = 'a3f602b65c0a076ea0d86079787f019b'

In [141]:
movies_dataset = pd.read_csv('./datasets/movies_dataset/movies_clear.csv')

In [142]:
ids = movies_dataset.id

In [161]:
def get_recommendations(item_id):
    movie = tmdb.Movies(item_id)
    recs = movie.recommendations()['results'][:7]
    return recs

In [163]:
movie_recs = pd.DataFrame(columns=['ID', 'Reco_Title', 'Reco_ID'])
i = 0
for item_id in ids:
    sims = get_recommendations(item_id)
    for sim in sims:
        movie_recs.loc[len(movie_recs.index)] = [item_id, sim['title'], sim['id']]

In [164]:
movie_recs

,ID,Reco_Title,Reco_ID
0,19995,The Avengers,24428
1,19995,The Dark Knight Rises,49026
2,19995,The Hobbit: An Unexpected Journey,49051
3,19995,The Hunger Games,70160
4,19995,Iron Man,1726
5,19995,Inception,27205
6,19995,Skyfall,37724
7,285,Pirates of the Caribbean: Dead Man's Chest,58
8,285,Pirates of the Caribbean: On Stranger Tides,1865
9,285,Pirates of the Caribbean: The Curse of the Bla...,22


In [242]:
(4800/8)/60

10.0

# Games

Ограничение: 8 запросов в секунду. Намного лучше, чем в Jikan

Twitch-токен нужно обновлять раз в n секунд. Пока держи это в голове.

In [286]:
import requests
import numpy as np
from igdb.wrapper import IGDBWrapper

In [183]:
url = 'https://id.twitch.tv/oauth2/token?client_id=x0rncah74zueqqov31fhkwuzlqv27a&client_secret=nyoj0usmdecwegm7e14c40s9kbq1hw&grant_type=client_credentials'

In [187]:
response = requests.request('POST', url=url)

In [201]:
access_token = response.text

In [202]:
access_token

'{"access_token":"wh0x0brf0afho4yix9x7k2gv735tgs","expires_in":5444906,"token_type":"bearer"}\n'

Количество секунд делится на количество секунд в 1 часе, умноженном на количество часов в сутках.

In [204]:
5444906/(3600*24)

63.01974537037037

То есть, токен нужно будет сменить через 64 суток. К тому времени, я уже закончу.

In [205]:
wrapper = IGDBWrapper('x0rncah74zueqqov31fhkwuzlqv27a', 'wh0x0brf0afho4yix9x7k2gv735tgs')

In [261]:
def get_game_item(item_id):
    byte_array = wrapper.api_request('games', 'fields *; where id = ' + str(item_id) + ';')
    decoded_bytes_array = byte_array.decode('utf-8')
    item = json.loads(decoded_bytes_array)
    return item

Количество уникальных значений делим на количество запросов в секунду и все это делим на количество секунд в 1 часе

In [238]:
(153838/8)/3600

5.341597222222222

И для этого набора, чтобы сделать полноценный набор данных, придется прождать 5 часов. Классненько.

Сейчас нужно сделать все, чтобы завтра я только подключил ассинхронные вычисления и все.

In [274]:
new_cols = ['storyline', 'summary']

In [284]:
cols = ['id', 'age_ratings', 'category', 'collection', 'external_games', 'first_release_date', 'game_modes', 'genres', 'involved_companies', 'keywords', 'name', 'platforms', 'rating', 'rating_count', 'similar_games', 'slug', 'tags', 'total_rating', 'total_rating_count']

In [15]:
4711/7

673.0

In [289]:
games_dataset = pd.DataFrame(columns=cols)
for i in range(1, 153838):
    item = get_game_item(i)[0]
    if len(item) == 0:
        continue
    for col in cols:
        try:
            games_dataset.loc[len(movie_recs.index)] = item[col]
        except KeyError:
            games_dataset[col] = np.NaN
        except ValueError:
            games_dataset[col] = str(item[col])
    if i > 20:
        break

In [290]:
games_dataset.head()

,id,age_ratings,category,collection,external_games,first_release_date,game_modes,genres,involved_companies,keywords,name,platforms,rating,rating_count,similar_games,slug,tags,total_rating,total_rating_count
0,21,28028,0,1,"[14839, 25458, 79006, 81698, 95265, 95266, 138...",1265673600,1,"[5, 12, 31]","[24, 25, 26, 28, 29, 20004, 42120, 42121, 4212...","[22, 67, 129, 138, 221, 226, 274, 301, 403, 45...",BioShock 2,"[6, 9, 12, 14]",80.921019,725,"[20, 41, 533, 538, 1020, 2031, 3188, 9498, 972...",bioshock-2,"[1, 18, 20, 268435461, 268435468, 268435487, 5...",83.960509,736
1,21,34379,0,1,"[14839, 25458, 79006, 81698, 95265, 95266, 138...",1265673600,2,"[5, 12, 31]","[24, 25, 26, 28, 29, 20004, 42120, 42121, 4212...","[22, 67, 129, 138, 221, 226, 274, 301, 403, 45...",BioShock 2,"[6, 9, 12, 14]",80.921019,725,"[20, 41, 533, 538, 1020, 2031, 3188, 9498, 972...",bioshock-2,"[1, 18, 20, 268435461, 268435468, 268435487, 5...",83.960509,736
